Primero leemos los ficheros iniciales de entrada.

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection


In [22]:
pcosData = pd.read_csv('./datos/pcos.csv', skiprows = 1, header = None,
                           names=['Age (yrs)', 'Weight (Kg)', 'Height(Cm)', 'BMI', 'Blood Group', 'Pulse rate(bpm)',
                                   'RR (breaths/min)', 'Hb(g/dl)', 'Cycle(R/I)', 'Cycle length(days)', 'Marriage Status (Yrs)',
                                     'Pregnant(Y/N)', 'No. of abortions', 'I beta-HCG(mIU/mL)', 'FSH(mIU/mL)', 'LH(mIU/mL)', 'FSH/LH', 'Hip(inch)',
                                       'Waist(inch)', 'Waist:Hip Ratio', 'TSH (mIU/L)', 'PRL(ng/mL)', 'Vit D3 (ng/mL)', 'PRG(ng/mL)', 'RBS(mg/dl)', 'Weight gain(Y/N)', 'hair growth(Y/N)',
                                         'Skin darkening (Y/N)', 'Hair loss(Y/N)', 'Pimples(Y/N)', 'Fast food (Y/N)', 'Reg.Exercise(Y/N)', 'BP _Systolic (mmHg)', 'BP _Diastolic (mmHg)', 'Follicle No. (L)',
                                           'Follicle No. (R)', 'Avg. F size (L) (mm)', 'Avg. F size (R) (mm)', 'Endometrium (mm)', 'PCOS (Y/N)'])


titanicData = pd.read_csv('./datos/titanic.csv', skiprows = 1, header = None,
                              names=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Initial', 'Age_band', 'Family_Size', 'Alone', 'Fare_cat', 'Deck', 'Title', 'Is_Married', 'Survived'])


Definimos la funcion para estandarizar los ficheros.

In [23]:

def dataStandarization (data, dataName, excludedColumns):
    if not isinstance (dataName, str):
        print("El argumento dataName debe ser una cadena de carateres.")
    
    elif not isinstance (excludedColumns, list):
        print("El argumento excludedColumns debe ser una lista con el nombre de las columnas a excluir.")

    else:
        
        data_copy = data.drop(excludedColumns, axis = 1)
        numerical_columns = [col for col in data.columns if col not in excludedColumns]
        columns_to_standarize = data[numerical_columns]

        scaler = StandardScaler()
        columns_to_standarize = scaler.fit_transform(columns_to_standarize)
        data[numerical_columns] = columns_to_standarize

        data.to_csv('./datos/' + dataName + '_standarized.csv', index = False)

        print("El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre " + dataName + ".csv")


Definimos las columnas que no vamos a considerar para la estandarización, en este caso las columnas booleanas y las columnas resultado.

In [24]:
titanic_excluded = ['Sex', 'Alone', 'Is_Married', 'Survived']
pcos_excluded = ['Pregnant(Y/N)', 'Weight gain(Y/N)', 'hair growth(Y/N)', 'Skin darkening (Y/N)', 'Hair loss(Y/N)', 'Pimples(Y/N)', 'Fast food (Y/N)','Reg.Exercise(Y/N)', 'PCOS (Y/N)']

dataStandarization(titanicData, 'titanic', titanic_excluded)
dataStandarization(pcosData, 'pcos', pcos_excluded)

El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre titanic.csv
El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre pcos.csv


A continuación dividiremos los ficheros estandarizados en dos, uno para el conjunto de entrenamiento y otro para el conjunto de prueba. Para ello definiremos la función necesaria.

In [25]:
def dataSplit (data, dataName):
    if not isinstance (dataName, str):
        print("El argumento dataName debe ser una cadena de carateres.")
    
    else:
        data_train, data_test = model_selection.train_test_split(data, test_size = 0.3, random_state = 99)
        data_train.to_csv('./datos/' + dataName + '_train.csv', index = False)
        data_test.to_csv('./datos/' + dataName + '_test.csv', index = False)

        print('Las dimensiones originales de los datos de entrada son: ', data.shape)
        print('El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre ' + dataName + '_train.csv, y sus dimensiones son: ', data_train.shape)
        print('El conjunto de pruebas se ha guardado en el directorio datos con el nombre ' + dataName + '_test.csv, y sus dimensiones son: ', data_test.shape)





Finalmente creamos los dos conjuntos de datos a partir del fichero estandarizado:

In [26]:
dataSplit(titanicData, 'titanic')
dataSplit(pcosData, 'pcos')

Las dimensiones originales de los datos de entrada son:  (891, 16)
El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre titanic_train.csv, y sus dimensiones son:  (623, 16)
El conjunto de pruebas se ha guardado en el directorio datos con el nombre titanic_test.csv, y sus dimensiones son:  (268, 16)
Las dimensiones originales de los datos de entrada son:  (541, 40)
El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre pcos_train.csv, y sus dimensiones son:  (378, 40)
El conjunto de pruebas se ha guardado en el directorio datos con el nombre pcos_test.csv, y sus dimensiones son:  (163, 40)


Una vez estandarizados los ficheros de entrenamientos, definiremos una función para que, dado un conjunto de datos, se generen diferentes conjuntos de datos de entrenamiento aplicando primero la técnica de Bootstrapping y más adelante Rndom Subspaces.

In [29]:

def generadorConjutosEntrenamiento(fileName, data, amountFiles):

    res= [] #lista de conjuntos de entrenamientos

    col = data[data.columns[:-1]]# cogemos todas las columnas salvo la ultima 
    
    objectiveVariable = data.loc[:, data.columns == data.columns[-1]] #guardamos la ultima columna

    numCol=col.shape[1] #dimension de la columna en col

    #hacemos bootstrapping
    for i in range (amountFiles):
        bootstrap_sample = data.sample(frac = 0.5, replace = True) 
        #para cada subset consideramos solo la mitad de los datos de entrada y con posibilidad de repeticion
        for j in range (amountFiles): #para cada nuevo archivo bootstrappeado
            #cogemos una serie de columnas aleatorias    
            selcted_column=np.random.choice(numCol,size=int(np.sqrt(numCol)),replace=False)#seleccionamos un subconjunto de columnas 
            subspace_sample = col.iloc[:, selcted_column].copy()#creamos un nuevo DataSet 
            #-----------------------------A LO MEJOR HAY QUE CAMBIARLO---------------
            subspace_sample[data.columns.values[-1]] = objectiveVariable #añadimos columna survived
            #------------------------------------------------------------------------

            res.append(subspace_sample)

            route = f'./datos/conjutosEntrenamiento/{fileName}_trainSet_{i+1}.{j+1}.csv'
            subspace_sample.to_csv(route, index = False)
            print('Fichero creados en la ruta: ', route)

    return res

Por último ejecutamos la función para ambos ficheros originales de entrenamiento, entroduciendo "titanic" y "pcos" respectivamente como "fileName" para mantener la coherencia de nombres en los archivos generados.

In [31]:
generadorConjutosEntrenamiento('titanic', titanicData, 2)
generadorConjutosEntrenamiento('pcos', pcosData, 2)


Fichero creados en la ruta:  ./datos/conjutosEntrenamiento/titanic_trainSet_1.1.csv
Fichero creados en la ruta:  ./datos/conjutosEntrenamiento/titanic_trainSet_1.2.csv
Fichero creados en la ruta:  ./datos/conjutosEntrenamiento/titanic_trainSet_2.1.csv
Fichero creados en la ruta:  ./datos/conjutosEntrenamiento/titanic_trainSet_2.2.csv
Fichero creados en la ruta:  ./datos/conjutosEntrenamiento/pcos_trainSet_1.1.csv
Fichero creados en la ruta:  ./datos/conjutosEntrenamiento/pcos_trainSet_1.2.csv
Fichero creados en la ruta:  ./datos/conjutosEntrenamiento/pcos_trainSet_2.1.csv
Fichero creados en la ruta:  ./datos/conjutosEntrenamiento/pcos_trainSet_2.2.csv


[     Vit D3 (ng/mL)  hair growth(Y/N)  LH(mIU/mL)  Marriage Status (Yrs)  \
 0         -0.094875                 0   -0.032219              -0.141744   
 1          0.032913                 0   -0.062129               0.692452   
 2         -0.000624                 0   -0.064554               0.483903   
 3         -0.047749                 0   -0.047462              -0.767391   
 4         -0.017682                 0   -0.064323              -1.393038   
 ..              ...               ...         ...                    ...   
 536       -0.038498                 0   -0.053814               0.066805   
 537       -0.077817                 0   -0.041919              -0.767391   
 538       -0.079263                 0   -0.042612               0.066805   
 539       -0.079552                 0   -0.024712              -1.184489   
 540       -0.094007                 1   -0.025059              -1.184489   
 
      Pulse rate(bpm)       BMI  PCOS (Y/N)  
 0           1.073680 -1.236